In [1]:
from experiment_new_ablation.gp_with_mean import BOWithMean
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from problems.photocatalysis_problems import *
from problems.standard_test_problems import *
from experiments_for_test_bos import check_model_test_problem

import time
import os

test_problems = [
    Ackley,
    Branin,
    Eggholder,
    GoldsteinPrice,
    SixHumpCamel,
    Shekel,
    Hartmann6,
    Michalewicz,
    Rosenbrock,
    StyblinskiTang
]

photocatalysis_problem, input_columns = get_kuka_problem()
bound = {}
for i in input_columns:
    bound[i] = (0, 5)
photocatalysis_problem.bound = bound
photocatalysis_problem.name = 'photocatalysis_experiment'

PATH = r'./experiment_new_ablation'
    
datetime = time.localtime()
folder_name = '{}_{}_{}_{}_{}'.format(str(datetime.tm_year), str(datetime.tm_mon), str(datetime.tm_mday),
                                      str(datetime.tm_hour), str(datetime.tm_min))
PATH = os.path.join(PATH, folder_name)

if not os.path.exists(PATH):
    os.mkdir(PATH)

In [2]:
def check_model_test_problem(problem, BO, max_iter=50, n_iter=100, save_result=True, path=PATH, acq='ei',
                             kappa=2.576, ml_regressor=None, use_noise=True):
    global regressor
    result_linear_custom = pd.DataFrame([])
    iter = 0
    path = PATH + '_' + acq + '_' + str(kappa)
    if not os.path.exists(path):
        os.makedirs(path)
    while iter < max_iter:
        if ml_regressor == RandomForestRegressor:
            regressor = ml_regressor(n_estimators=20, max_depth=5)
            name = 'RFBO'
        elif ml_regressor == LinearRegression:
            regressor = ml_regressor()
            name = 'LinearBO'
        else:
            regressor = None
            name = 'standardBO'
        optimizer = BO(f=problem, pbounds=problem.bound, ml_regressor=regressor, use_noise=use_noise, random_state=iter)

        optimizer.maximize(n_iter=n_iter, acq=acq, kappa=kappa, init_points=5)
        result_linear_custom = result_linear_custom.append(pd.Series(optimizer.result_dataframe, dtype=np.float64),
                                                           ignore_index=True)
        iter += 1
    if save_result:
        result_linear_custom = pd.DataFrame(np.array(result_linear_custom))
        result_linear_custom.to_csv(
            os.path.join(path, '{}_result_test_{}.csv'.format(name, problem.name)))

In [3]:
check_model_test_problem(photocatalysis_problem, BOWithMean, ml_regressor=LinearRegression, use_noise=True, acq='ucb')

|   iter    |  target   | AcidRe... | L-Cyst... | Methyl... | NaCl-3... | NaOH-1... | P10-MI... | PVP-1w... | Rhodam... | SDS-1w... | Sodium... | water_... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  6.204    |  2.744    |  3.576    |  3.014    |  2.724    |  2.118    |  3.229    |  2.188    |  4.459    |  4.818    |  1.917    |  3.959    |
|  2        |  5.7      |  2.644    |  2.84     |  4.628    |  0.3552   |  0.4356   |  0.1011   |  4.163    |  3.891    |  4.35     |  4.893    |  3.996    |
|  3        |  6.091    |  2.307    |  3.903    |  0.5914   |  3.2      |  0.7168   |  4.723    |  2.609    |  2.073    |  1.323    |  3.871    |  2.281    |
|  4        |  4.123    |  2.842    |  0.09395  |  3.088    |  3.06     |  3.085    |  4.719    |  3.409    |  1.798    |  2.185    |  3.488    |  0.3011   |
|  5        |  3.917    |  3.334    |  3.353    |  1

C:\Users\darkn\anaconda3\lib\site-packages\numpy\core\_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


TypeError: unsupported operand type(s) for +: 'float' and 'dict'